In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flight-fare-prediction-mh/Data_Train.xlsx
/kaggle/input/flight-fare-prediction-mh/Sample_submission.xlsx
/kaggle/input/flight-fare-prediction-mh/Test_set.xlsx


In [2]:
#importing dependencies 
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [3]:
train = pd.read_excel('/kaggle/input/flight-fare-prediction-mh/Data_Train.xlsx') #loading data
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [4]:
test = pd.read_excel('/kaggle/input/flight-fare-prediction-mh/Test_set.xlsx')
test.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          2671 non-null   object
 1   Date_of_Journey  2671 non-null   object
 2   Source           2671 non-null   object
 3   Destination      2671 non-null   object
 4   Route            2671 non-null   object
 5   Dep_Time         2671 non-null   object
 6   Arrival_Time     2671 non-null   object
 7   Duration         2671 non-null   object
 8   Total_Stops      2671 non-null   object
 9   Additional_Info  2671 non-null   object
dtypes: object(10)
memory usage: 208.8+ KB


In [7]:
# checking null values
train.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [8]:
#combining the data together
final_data = train.append(test)

In [9]:
final_data.sample(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
7744,IndiGo,21/06/2019,Banglore,Delhi,BLR → DEL,23:30,02:20 22 Jun,2h 50m,non-stop,No info,4077.0
1110,IndiGo,01/03/2019,Banglore,New Delhi,BLR → STV → DEL,11:20,15:25,4h 5m,1 stop,No info,NaN
3302,SpiceJet,21/06/2019,Kolkata,Banglore,CCU → BLR,22:20,00:40 22 Jun,2h 20m,non-stop,No info,3873.0


In [10]:
final_data.shape

(13354, 11)

In [11]:
#feature engineering process on Date_of_Journey

final_data['date']=final_data['Date_of_Journey'].apply(lambda x:x.split('/')[0])
final_data['month']=final_data['Date_of_Journey'].apply(lambda x:x.split('/')[1])
final_data['year']=final_data['Date_of_Journey'].apply(lambda x:x.split('/')[2])


In [12]:
final_data['date']=final_data['date'].astype(int)
final_data['month']=final_data['month'].astype(int)
final_data['year']=final_data['year'].astype(int)

In [13]:
final_data.drop('Date_of_Journey',axis=1,inplace = True)


In [14]:
final_data.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,date,month,year
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,5,2019
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,9,6,2019
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,12,5,2019
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,1,3,2019


In [15]:
final_data['Arrival_Time']=final_data['Arrival_Time'].apply(lambda x:x.split(' ')[0])

In [16]:
final_data['Arrival_hour']=final_data['Arrival_Time'].apply(lambda x:x.split(':')[0])
final_data['Arrival_min']=final_data['Arrival_Time'].apply(lambda x:x.split(':')[1])
final_data['Arrival_hour']=final_data['Arrival_hour'].astype(int)
final_data['Arrival_min']=final_data['Arrival_min'].astype(int)

In [17]:
#droping Arrival_time 
final_data.drop('Arrival_Time',axis=1,inplace = True)

In [18]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Source           13354 non-null  object 
 2   Destination      13354 non-null  object 
 3   Route            13353 non-null  object 
 4   Dep_Time         13354 non-null  object 
 5   Duration         13354 non-null  object 
 6   Total_Stops      13353 non-null  object 
 7   Additional_Info  13354 non-null  object 
 8   Price            10683 non-null  float64
 9   date             13354 non-null  int64  
 10  month            13354 non-null  int64  
 11  year             13354 non-null  int64  
 12  Arrival_hour     13354 non-null  int64  
 13  Arrival_min      13354 non-null  int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 1.5+ MB


In [19]:
final_data['Dep_hour']=final_data['Dep_Time'].apply(lambda x:x.split(':')[0])
final_data['Dep_min']=final_data['Dep_Time'].apply(lambda x:x.split(':')[1])
final_data['Dep_hour']=final_data['Dep_hour'].astype(int)
final_data['Dep_min']=final_data['Dep_min'].astype(int)

In [20]:
#droping Dep_time 
final_data.drop('Dep_Time',axis=1,inplace = True)

In [21]:
final_data.head(2)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,date,month,year,Arrival_hour,Arrival_min,Dep_hour,Dep_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662.0,1,5,2019,13,15,5,50


In [22]:
final_data['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [23]:
final_data['Total_Stops']=final_data['Total_Stops'].map({'non-stop':0,'2 stops':2,'1 stop':1,'3 stops':3,'4 stops':4,'nan':1})

In [24]:
final_data.head(3)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,date,month,year,Arrival_hour,Arrival_min,Dep_hour,Dep_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,0.0,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2.0,No info,7662.0,1,5,2019,13,15,5,50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2.0,No info,13882.0,9,6,2019,4,25,9,25


In [25]:
#droping the route column because not required
final_data.drop('Route',axis=1,inplace =True)

In [26]:
final_data['duration_hour']=final_data['Duration'].str.split(' ').str[0].str.split('h').str[0]


final_data[final_data['duration_hour']=='5m']

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,date,month,year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,duration_hour
6474,Air India,Mumbai,Hyderabad,5m,2.0,No info,17327.0,6,3,2019,16,55,16,50,5m
2660,Air India,Mumbai,Hyderabad,5m,2.0,No info,NaN,12,3,2019,16,55,16,50,5m


In [27]:
final_data.drop(6474,axis=0,inplace= True)
final_data.drop(2660,axis=0,inplace= True)

In [28]:
final_data.sample(2)

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,date,month,year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,duration_hour
999,Jet Airways,Kolkata,Banglore,7h 30m,1.0,In-flight meal not included,9663.0,24,5,2019,4,40,21,10,7
4042,Air India,Banglore,Delhi,2h 50m,0.0,No info,5176.0,6,4,2019,16,10,13,20,2


In [29]:
final_data['duration_hour']=final_data['duration_hour'].astype(int)
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13351 entries, 0 to 2670
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13351 non-null  object 
 1   Source           13351 non-null  object 
 2   Destination      13351 non-null  object 
 3   Duration         13351 non-null  object 
 4   Total_Stops      13350 non-null  float64
 5   Additional_Info  13351 non-null  object 
 6   Price            10681 non-null  float64
 7   date             13351 non-null  int64  
 8   month            13351 non-null  int64  
 9   year             13351 non-null  int64  
 10  Arrival_hour     13351 non-null  int64  
 11  Arrival_min      13351 non-null  int64  
 12  Dep_hour         13351 non-null  int64  
 13  Dep_min          13351 non-null  int64  
 14  duration_hour    13351 non-null  int64  
dtypes: float64(2), int64(8), object(5)
memory usage: 1.6+ MB


In [30]:
final_data['duration_time_in_min'] = final_data['duration_hour']*60 

In [31]:
final_data.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,date,month,year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,duration_hour,duration_time_in_min
0,IndiGo,Banglore,New Delhi,2h 50m,0.0,No info,3897.0,24,3,2019,1,10,22,20,2,120
1,Air India,Kolkata,Banglore,7h 25m,2.0,No info,7662.0,1,5,2019,13,15,5,50,7,420
2,Jet Airways,Delhi,Cochin,19h,2.0,No info,13882.0,9,6,2019,4,25,9,25,19,1140
3,IndiGo,Kolkata,Banglore,5h 25m,1.0,No info,6218.0,12,5,2019,23,30,18,5,5,300
4,IndiGo,Banglore,New Delhi,4h 45m,1.0,No info,13302.0,1,3,2019,21,35,16,50,4,240


In [32]:
final_data.drop(['Duration','duration_hour'],axis=1,inplace=True)

In [33]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13351 entries, 0 to 2670
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Airline               13351 non-null  object 
 1   Source                13351 non-null  object 
 2   Destination           13351 non-null  object 
 3   Total_Stops           13350 non-null  float64
 4   Additional_Info       13351 non-null  object 
 5   Price                 10681 non-null  float64
 6   date                  13351 non-null  int64  
 7   month                 13351 non-null  int64  
 8   year                  13351 non-null  int64  
 9   Arrival_hour          13351 non-null  int64  
 10  Arrival_min           13351 non-null  int64  
 11  Dep_hour              13351 non-null  int64  
 12  Dep_min               13351 non-null  int64  
 13  duration_time_in_min  13351 non-null  int64  
dtypes: float64(2), int64(8), object(4)
memory usage: 1.5+ MB


In [34]:
final_data['Airline'].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [35]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
final_data['Airline']= labelencoder.fit_transform(final_data['Airline'])
final_data['Source']= labelencoder.fit_transform(final_data['Source'])
final_data['Destination']= labelencoder.fit_transform(final_data['Destination'])
final_data['Additional_Info']= labelencoder.fit_transform(final_data['Additional_Info'])


In [36]:
final_data = pd.get_dummies(final_data,columns=["Airline", "Source", "Destination",'Additional_Info'] ,drop_first = True)

In [37]:
final_data

,Total_Stops,Price,date,month,year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,duration_time_in_min,...,Destination_5,Additional_Info_1,Additional_Info_2,Additional_Info_3,Additional_Info_4,Additional_Info_5,Additional_Info_6,Additional_Info_7,Additional_Info_8,Additional_Info_9
0,0.0,3897.0,24,3,2019,1,10,22,20,120,...,1,0,0,0,0,0,0,0,1,0
1,2.0,7662.0,1,5,2019,13,15,5,50,420,...,0,0,0,0,0,0,0,0,1,0
2,2.0,13882.0,9,6,2019,4,25,9,25,1140,...,0,0,0,0,0,0,0,0,1,0
3,1.0,6218.0,12,5,2019,23,30,18,5,300,...,0,0,0,0,0,0,0,0,1,0
4,1.0,13302.0,1,3,2019,21,35,16,50,240,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,1.0,NaN,6,6,2019,20,25,20,30,1380,...,0,0,0,0,0,0,0,0,1,0
2667,0.0,NaN,27,3,2019,16,55,14,20,120,...,0,0,0,0,0,0,0,0,1,0
2668,1.0,NaN,6,3,2019,4,25,21,50,360,...,0,0,0,0,0,0,0,0,1,0
2669,1.0,NaN,6,3,2019,19,15,4,0,900,...,0,0,0,0,0,0,0,0,1,0
